In [38]:
from section_divide import _get_section
from arrange_score import get_notes_score
from pprint import pprint
import matplotlib.pyplot as plt
import os
import numpy as np
from classes.types import HoldType, JudgeType, NotesType
from classes.types.HoldType import HoldType

notes_score = get_notes_score("./data/m155.json")
for note in notes_score:
    if abs(note.y - 97.25) < 0.0001:
        print(note)
        break


sections = _get_section("./data/m155.json")
# for note in sections[257]:
#     print(note)

assert (sections[257][0].type ) == NotesType.HOLD, f"{sections[257][0]}"
dir_path = f"./data/_vis-result/notes"
# pprint(sections)
try:
    os.mkdir(dir_path)
except FileExistsError:
    pass

# max_duration = 0
# for section in sections:
#     max_duration = max(max_duration, section[-1].y - section[0].y)

for i,section in enumerate(sections):
    dir = f"{dir_path}/length-{len(section)}"
    try:
        os.mkdir(dir)
    except FileExistsError:
        pass
    
    ys = np.array([note.y for note in section])
    min_y = ys.min()
    
    # normal
    xs = np.array([note.x for note in section if note.type != JudgeType.HOLD])
    ys = np.array([note.y for note in section if note.type != JudgeType.HOLD], dtype="float64")
    # ys -= min_y
    plt.scatter(xs,ys, color="b")
    
    hold_x = []
    hold_y = []
    for note in section:
        if note.type != JudgeType.HOLD:
            continue
        
        if note.hold_type == HoldType.END:
            # plt.plot(hold_x, np.array(hold_y) - min_y, color="g",  marker="o")
            plt.plot(hold_x, hold_y, color="g",  marker="o")
            hold_x = []
            hold_y = []
            continue
        
        hold_x.append(note.x)
        hold_y.append(note.y)
    
    # # hold
    # xs = np.array([note.x for note in section if note.type == NotesType.HOLD])
    # ys = np.array([note.y for note in section if note.type == NotesType.HOLD], dtype="float64")
    # ys -= min_y
    # plt.plot(xs, ys, color="g",  marker="o")
    
    

    plt.xlim([-0.3,12.3])
    # plt.tick_params(labelbottom=False, labelleft=False, labelright=False, labeltop=False, bottom=False, left=False, right=False, top=False)
    plt.savefig(f"{dir}/{i}.png")
    plt.close()
    if len(section) == 66:
        for note in section:
            print(note)
    
    # break


Note describe:
	pos: (x, y) = (1, 97.25)
	width: 5
	note type: FLICK 
	how judge: OFF


AssertionError: Note describe:
	pos: (x, y) = (1, 97.25)
	width: 5
	note type: FLICK 
	how judge: OFF